# Entrenamiento de un LLM

En este notebook se incorpora un ejemplo de entrenamiento de un LLM haciendo uso de una técnica conocida como LowRank para hacer este entrenamiento posible en el hardware disponible, ya que permite entrenar un número muy reducido de nuevos pesos, en comparación con el número de pesos total del modelo. 

## Instalación e importación de librerías

In [ ]:
!pip install transformers torch datasets peft -q

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AutoModelForSeq2SeqLM, TrainingArguments
import torch
from transformers import Trainer , AutoTokenizer, AutoModelForCausalLM
from peft import get_peft_model, PeftModel, PeftConfig, AutoPeftModelForCausalLM, LoraConfig, TaskType

## Obtención de modelo y tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("deepset/tinyroberta-squad2")
model = AutoModelForCausalLM.from_pretrained("deepset/tinyroberta-squad2")

## Aplicación de LORA para evitar entrenar todo el modelo

In [ ]:
peft_config = LoraConfig(task_type=TaskType.CAUSAL_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

## Obtención del dataset

In [ ]:
from datasets import load_dataset

# Cargar el dataset
ds = load_dataset("Malikeh1375/medical-question-answering-datasets", "all-processed", split='train')

# Seleccionar solo los primeros 100 elementos
ds = ds.select(range(100))

In [ ]:
# Verifica el tamaño del nuevo dataset
print(len(ds))  # Debería imprimir 100

In [ ]:
def preprocess_function(examples):
    # Concatenar 'instruction' con 'input' para formar la entrada del modelo.
    inputs = examples['instruction'] + " " + examples['input']  # Ajustar si la concatenación es diferente
    outputs = examples['output']  # La salida es la columna 'output'

    # Tokenizar la entrada (inputs)
    model_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=512)

    # Tokenizar la salida (outputs)
    with tokenizer.as_target_tokenizer():  # Asegurarse de que el tokenizador procese bien las etiquetas
        labels = tokenizer(outputs, padding="max_length", truncation=True, max_length=128)

    # Reemplazar los tokens de padding en los labels por -100 (ignorados en la pérdida)
    labels["input_ids"] = [(label if label != tokenizer.pad_token_id else -100) for label in labels["input_ids"]]

    # Añadir las etiquetas al diccionario de las entradas
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

# Preprocesar el dataset tokenizándolo
tokenized_ds = ds.map(preprocess_function, batched=False)

## Definición del bucle de entrenamiento

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=1e-3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [ ]:
from transformers import Trainer

# Definir el Trainer con el dataset ya tokenizado
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds,  # Usamos el dataset tokenizado
    tokenizer=tokenizer  # Tokenizador para asegurarse de que el modelo está alineado con los tokens
)

# Entrenar el modelo
trainer.train()